In [7]:
import sys
import os
import pandas as pd
import numpy as np
import datetime as dt
import h5py
import json
import requests
import base64
sys.path.append('../')

pd.options.display.max_rows=99

In [8]:
github_userName = 'Tanag3r'
ebird_token = 'j6c7l80ga2ib'

In [17]:
##pull in trailhead stops
trailheadRef = pd.DataFrame()
trailheadRef_json = 'https://raw.githubusercontent.com/Tanag3r/trailheadDirect_birds/main/trailheadsRef.json'
response = requests.get(trailheadRef_json)
trailheadRef = pd.DataFrame(response.json())
##trailheadRef.set_index('StopName', inplace=True)

Route         object
StopName      object
Address       object
Latitude     float64
Longitude    float64
dtype: object

In [21]:
##set types for error checking, other cleaning
trailheadRef['Latitude'] = trailheadRef['Latitude'].fillna(0.00)
trailheadRef['Longitude'] = trailheadRef['Longitude'].fillna(0.00)
trailheadRef['Latitude'] = trailheadRef['Latitude'].astype('float64')
trailheadRef['Longitude'] = trailheadRef['Longitude'].astype('float64')
trailheadRef['GeoTest'] = ''
trailheadRef['GeoTest'] = trailheadRef['GeoTest'].astype('str')
##check length of lat and long, each must be at least 4 char
for i in trailheadRef:
    latLen = len(trailheadRef['Latitude'])
    lngLen = len(trailheadRef['Longitude'])
    address = trailheadRef['Address']
    trailheadRef['GeoTest'] = trailheadRef['GeoTest'].astype('str')
    if (latLen + lngLen) < 4 or address.empty:
        trailheadRef['GeoTest'] = 'Fail'
    trailheadRef['GeoTest'] = 'Pass'

##TODO: fetch coordinates of trailheads using addresses
##while trailheadRef['GeoTest'] is 'Fail':
    ##do the thing
trailheadRef.head()

,Route,StopName,Address,Latitude,Longitude,GeoTest
0,IssaquahAlps,EastSunsetWay,"661-831 E Sunset Way, Issaquah, WA 98027",47.529635,-122.025119,Pass
1,IssaquahAlps,HighSchool,"Parking lot, The Rainier Trail, Issaquah, WA 9...",47.519345,-122.029801,Pass
2,IssaquahAlps,ChiricoTrail_PooPooPoint,"11400 Issaquah-Hobart Road Southeast, Issaquah...",47.499949,-122.021730,Pass
3,IssaquahAlps,SquakMountain,"13201 Squak Mountain Rd SE, Issaquah, WA 98027",47.481465,-122.053997,Pass
4,IssaquahAlps,MargaretsWay,"190th Ave SE, Issaquah, WA 98027",47.506620,-122.086660,Pass


In [10]:
##fetch all hotspots within 4 kilometers of each trailhead

trailheadHotspots = []

for StopName in trailheadRef.itertuples():
    url = '''https://api.ebird.org/v2/ref/hotspot/geo?lat={}&lng={}&dist=4&fmt=json'''.format(StopName.Latitude,StopName.Longitude)
    ebirdapi_auth_header = {'X-eBirdApiToken': ebird_token}
    ebird_request = requests.get(url,headers=ebirdapi_auth_header)
    ebird_response = pd.DataFrame(ebird_request.json())
    if ebird_request.status_code == requests.codes.ok:
        ebird_response['StopName'] = StopName.StopName
        trailheadHotspots.append(ebird_response)
    ebird_request.raise_for_status()

In [11]:
all_trailheadHotspots = pd.concat(trailheadHotspots)

,locId,locName,countryCode,subnational1Code,subnational2Code,lat,lng,latestObsDt,numSpeciesAllTime,StopName
0,L10128988,Berntsen Park,US,US-WA,US-WA-033,47.538439,-122.043281,2021-12-15 11:46,15.0,EastSunsetWay
1,L10129014,"Confluence Park, Issaquah",US,US-WA,US-WA-033,47.536469,-122.039342,2022-01-08 12:54,44.0,EastSunsetWay
2,L8365620,Issaquah High School,US,US-WA,US-WA-033,47.522490,-122.028687,2021-12-17 08:00,58.0,EastSunsetWay
3,L8102503,Issaquah Salmon Hatchery,US,US-WA,US-WA-033,47.529503,-122.039512,2021-12-03 08:22,33.0,EastSunsetWay
4,L3352189,"Maple Street ponds, Issaquah",US,US-WA,US-WA-033,47.543027,-122.053715,2021-11-30 10:12,46.0,EastSunsetWay
5,L4920968,Pickering Place Pond,US,US-WA,US-WA-033,47.548110,-122.052462,2022-01-08 16:02,69.0,EastSunsetWay
6,L11570424,Pickering Place trails,US,US-WA,US-WA-033,47.550100,-122.049440,2021-08-31 15:56,51.0,EastSunsetWay
7,L10129002,Salmon Run Nature Park,US,US-WA,US-WA-033,47.539272,-122.043434,2021-12-18 09:02,18.0,EastSunsetWay
8,L854460,Squak Mountain SP,US,US-WA,US-WA-033,47.510302,-122.047720,2021-09-15 16:17,72.0,EastSunsetWay
9,L1167737,Tibbets Valley Park,US,US-WA,US-WA-033,47.539340,-122.061066,2021-12-18 11:57,45.0,EastSunsetWay


In [12]:
all_trailheadHotspots.shape

(111, 10)

In [13]:
##Be nice to eBirds and print the results
today = dt.datetime.today()
filename = 'trailheadHotspots_Y{}_M{}_D{}.csv'.format(today.year,today.month,today.day)

all_trailheadHotspots.to_csv(filename,sep=',',index=False)